In [ ]:
#| default_exp evaluation.benchmarks

# Evaluation Benchmarks

> Setup for evaluation datasets and DataLoaders. Defines Indic-specific categories and provides functions to load benchmark data.

## Colab Setup

In [ ]:
#| hide
# Mount Google Drive (Optional, but recommended for persistent storage)
from pathlib import Path
import sys
import os

try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
    # Define PROJECT_ROOT for Colab
    PROJECT_ROOT = Path('/content/drive/MyDrive/Indic-Clip') # Adjust path if needed
    if not PROJECT_ROOT.exists():
        print(f"Warning: Project directory not found at {PROJECT_ROOT}. Please ensure it exists.")
    else:
        # Add project root to sys.path
        if str(PROJECT_ROOT) not in sys.path:
            sys.path.insert(0, str(PROJECT_ROOT))
            print(f"Added {PROJECT_ROOT} to sys.path")
        # Change current working directory
        os.chdir(PROJECT_ROOT)
        print(f"Changed working directory to: {os.getcwd()}")

except ModuleNotFoundError:
    print("Not running in Colab, skipping Drive mount.")
    # Define PROJECT_ROOT for local execution (adjust if needed)
    PROJECT_ROOT = Path.cwd()
    if PROJECT_ROOT.name == 'nbs': PROJECT_ROOT = PROJECT_ROOT.parent
    print(f"Running locally. Project root assumed: {PROJECT_ROOT}")
    if str(PROJECT_ROOT) not in sys.path:
         sys.path.insert(0, str(PROJECT_ROOT))
         print(f"Added {PROJECT_ROOT} to sys.path")
except Exception as e:
    print(f"An error occurred during Colab setup: {e}")
    PROJECT_ROOT = Path('.').resolve()
    print(f"Defaulting project root to current dir: {PROJECT_ROOT}")

In [ ]:
#| hide
# Install requirements if needed (especially in Colab)
# !pip install -qr requirements.txt
# !pip install scikit-learn # Ensure sklearn is installed

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch
import logging
import json
import pandas as pd
from pathlib import Path
from typing import List, Dict, Union, Callable

from fastai.vision.all import (
    DataBlock, ImageBlock, CategoryBlock, ColReader, RandomSplitter,
    Resize, Normalize, imagenet_stats, DataLoader, Datasets, TfmdLists
)
from fastcore.all import *

# --- Project Imports ---
try:
    from indic_clip.core import (
        get_logger, setup_logging, BENCHMARK_DATA_PATH, DEFAULT_IMAGE_SIZE,
        HINDI_RAW_PATH, SANSKRIT_RAW_PATH # Needed if benchmark images are in raw dirs
    )
    from indic_clip.data.creation import ImageGetter # Reuse image getter if structure is same
except ModuleNotFoundError:
    print("Could not import project modules in 12_evaluation_benchmarks.ipynb. Using fallbacks.")
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    def get_logger(name): return logging.getLogger(name)
    def setup_logging(): pass
    BENCHMARK_DATA_PATH = Path('./data/benchmarks')
    DEFAULT_IMAGE_SIZE = 224
    HINDI_RAW_PATH = Path('./data/raw/hindi')
    SANSKRIT_RAW_PATH = Path('./data/raw/sanskrit')
    # Dummy ImageGetter if needed
    def ImageGetter(row): return BENCHMARK_DATA_PATH / 'dummy_image.jpg'

logger = get_logger(__name__)

## Indic-Specific Categories

In [ ]:
#| export
# --- Example Indic Categories for Zero-Shot Classification ---
# These lists should be curated based on expected evaluation domains.
# Hindi
HINDI_RELIGIOUS_FIGURES = [
    "कृष्ण", "राम", "शिव", "गणेश", "हनुमान", "दुर्गा", "लक्ष्मी", "सरस्वती", "विष्णु", "बुद्ध", "महावीर"
]
HINDI_ECOMMERCE_CATEGORIES = [
    "साड़ी", "कुर्ता", "मोबाइल फोन", "जूते", "किताब", "घड़ी", "चश्मा", "हेडफ़ोन", "लैपटॉप", "कैमरा", "गहने"
]

# Sanskrit (Example - adjust based on available data/domain)
SANSKRIT_RELIGIOUS_CONCEPTS = [
    "देव", "देवी", "यज्ञ", "मन्त्र", "वेद", "उपनिषद्", "पुराण", "योग", "ध्यान", "धर्म", "कर्म"
]

# Combine or select based on evaluation needs
DEFAULT_ZS_CATEGORIES_HI = HINDI_RELIGIOUS_FIGURES + HINDI_ECOMMERCE_CATEGORIES
DEFAULT_ZS_CATEGORIES_SA = SANSKRIT_RELIGIOUS_CONCEPTS

# --- Example Prompt Templates --- (Use appropriate language)
DEFAULT_PROMPT_TEMPLATES_HI = [
    "यह {} की तस्वीर है।",
    "{} का एक चित्र।",
    "एक फोटो जिसमें {} दिखाया गया है।"
]
DEFAULT_PROMPT_TEMPLATES_SA = [
    "इदं {} चित्रम् अस्ति।",
    "{} एकं चित्रम्।"
]
DEFAULT_PROMPT_TEMPLATES_EN = [
    "a photo of a {}",
    "a picture of {}",
    "an image depicting {}",
    "{} shown in an image"
]

## Benchmark Loading Functions

In [ ]:
#| export
def load_benchmark_data(benchmark_name: str, base_path: Path = BENCHMARK_DATA_PATH) -> pd.DataFrame | None:
    """Loads image paths and labels for a specific benchmark dataset.
       Assumes a standard format (e.g., CSV/JSONL) within the benchmark directory.

    Args:
        benchmark_name (str): Name of the benchmark (e.g., 'imagenet_val', 'flickr30k_hi').
        base_path (Path): Base directory containing benchmark subdirectories.

    Returns:
        pd.DataFrame: DataFrame with columns like 'image_path', 'label' (or 'captions'), 'split'.
                      Returns None if loading fails or format is unknown.
    """
    benchmark_dir = base_path / benchmark_name
    logger.info(f"Attempting to load benchmark data from: {benchmark_dir}")

    if not benchmark_dir.is_dir():
        logger.error(f"Benchmark directory not found: {benchmark_dir}")
        return None

    # --- Example Loading Logic (Adapt based on actual benchmark formats) ---

    # Example 1: CSV format (e.g., image_path,label_id,split)
    csv_file = benchmark_dir / f"{benchmark_name}.csv"
    if csv_file.exists():
        try:
            df = pd.read_csv(csv_file)
            # Basic validation (adjust columns as needed)
            if 'image_path' not in df.columns or ('label' not in df.columns and 'caption' not in df.columns):
                logger.error(f"CSV {csv_file} missing required columns ('image_path' and 'label'/'caption'). Found: {df.columns}")
                return None
            logger.info(f"Loaded {len(df)} items from CSV: {csv_file}")
            return df
        except Exception as e:
            logger.error(f"Error loading CSV benchmark data from {csv_file}: {e}")
            return None

    # Example 2: JSONL format (e.g., {'image_path': '...', 'label': '...', 'split': '...'})
    jsonl_file = benchmark_dir / f"{benchmark_name}.jsonl"
    if jsonl_file.exists():
        data = []
        try:
            with open(jsonl_file, 'r', encoding='utf-8') as f:
                for line in f:
                    item = json.loads(line)
                    # Add validation for required keys
                    if 'image_path' in item and ('label' in item or 'caption' in item):
                        data.append(item)
                    else:
                        logger.warning(f"Skipping invalid item in {jsonl_file}: {item}")
            df = pd.DataFrame(data)
            logger.info(f"Loaded {len(df)} items from JSONL: {jsonl_file}")
            return df
        except Exception as e:
            logger.error(f"Error loading JSONL benchmark data from {jsonl_file}: {e}")
            return None

    # --- Add more loading logic for other formats --- (e.g., ImageFolder structure)

    logger.error(f"Could not find suitable data file (CSV/JSONL) in {benchmark_dir} for benchmark '{benchmark_name}'.")
    return None

## DataLoader Creation Functions

In [ ]:
#| export
def create_zeroshot_dataloader(
    df: pd.DataFrame,
    image_path_col: str = 'image_path',
    label_col: str = 'label',
    benchmark_base_path: Path = BENCHMARK_DATA_PATH,
    img_size: int = DEFAULT_IMAGE_SIZE,
    batch_size: int = 64,
    num_workers: int = 4
) -> DataLoader | None:
    """Creates a DataLoader for zero-shot classification evaluation.

    The DataLoader yields tuples of (image_tensor, label_index).

    Args:
        df (pd.DataFrame): DataFrame containing image paths and corresponding labels (indices).
        image_path_col (str): Name of the column containing relative image paths.
        label_col (str): Name of the column containing ground truth labels (as indices).
        benchmark_base_path (Path): The absolute base path where the benchmark images are stored.
                                      The paths in df[image_path_col] are assumed relative to this.
        img_size (int): Target image size.
        batch_size (int): DataLoader batch size.
        num_workers (int): Number of workers for the DataLoader.

    Returns:
        DataLoader: A fastai DataLoader for evaluation, or None if columns are missing.
    """
    if image_path_col not in df.columns or label_col not in df.columns:
        logger.error(f"Required columns '{image_path_col}' or '{label_col}' not found in DataFrame.")
        return None

    # Function to construct the full image path
    def get_img_path(row): return benchmark_base_path / row[image_path_col]

    # Define a simple DataBlock for loading images and labels
    # No splitting needed as we evaluate on the whole set (usually test set)
    # Use TfmdLists directly for more control over evaluation datasets

    # 1. Create item transforms (Resize)
    item_tfms = Resize(img_size)
    # 2. Create batch transforms (Normalize)
    batch_tfms = Normalize.from_stats(*imagenet_stats)

    # 3. Create Datasets
    # We need image paths and integer labels
    img_paths = df.apply(get_img_path, axis=1).tolist()
    labels = df[label_col].tolist()

    # Ensure labels are integers
    try:
        labels = [int(l) for l in labels]
    except ValueError:
        logger.error(f"Could not convert all labels in column '{label_col}' to integers.")
        return None

    # Create TfmdLists: applies item_tfms lazily
    # Input pipeline: Path -> Image -> PILImage
    # Target pipeline: int -> TensorCategory
    tls = TfmdLists(zip(img_paths, labels),
                    tfms=[[PILImage.create], [CategoryEncode()]]) # No item_tfms here yet

    # Apply item transforms to the input part (images)
    tls = tls.map_tfms([item_tfms, None], dl_type=Datasets) # None means no tfms for labels


    # 4. Create DataLoader
    # `after_item` applies transforms to individual items after they are fetched
    # `before_batch` applies transforms before collation
    # `after_batch` applies transforms to the collated batch (like normalization)
    dls = DataLoader(tls, bs=batch_size, num_workers=num_workers,
                     after_item=[ToTensor], # Convert PILImage to Tensor
                     after_batch=[IntToFloatTensor, batch_tfms], # Normalize batch
                     shuffle=False, drop_last=False) # No shuffle/drop for validation

    logger.info(f"Created Zero-Shot DataLoader with {len(tls)} items.")
    return dls

In [ ]:
#| export
# Placeholder for Retrieval DataLoader - Adapt from 04_data_creation if needed
# This might be necessary if retrieval benchmarks have a different structure
# or require specific test splits.

# def create_retrieval_dataloader(...):
#     logger.info("Retrieval DataLoader creation not fully implemented yet.")
#     # Reuse IndicCLIPDataBlock logic from 04_data_creation if applicable,
#     # possibly filtering the source DataFrame for the specific benchmark and split.
#     # Ensure shuffle=False and drop_last=False for evaluation.
#     return None

## Example Usage

In [ ]:
#| eval: false
# Example: Load data and create a Zero-Shot DataLoader
if __name__ == '__main__':
    print("--- Running Evaluation Benchmark Setup Example ---")

    # 1. Define benchmark specifics
    benchmark_name = "dummy_zeroshot_benchmark" # Replace with actual benchmark name
    benchmark_path = BENCHMARK_DATA_PATH
    ensure_dir(benchmark_path / benchmark_name)

    # 2. Create dummy benchmark data file (CSV example)
    dummy_csv_path = benchmark_path / benchmark_name / f"{benchmark_name}.csv"
    dummy_data = {
        'image_path': [f'images/img_{i}.jpg' for i in range(10)],
        'label': [0, 1, 2, 0, 1, 2, 0, 1, 2, 0] # Labels are indices into class_names
    }
    pd.DataFrame(dummy_data).to_csv(dummy_csv_path, index=False)
    print(f"Created dummy benchmark CSV: {dummy_csv_path}")
    # Create dummy image directory and files
    dummy_image_dir = benchmark_path / benchmark_name / 'images'
    ensure_dir(dummy_image_dir)
    for i in range(10):
        try:
            img = Image.new('RGB', (60, 30), color = 'red') # Create small dummy images
            img.save(dummy_image_dir / f'img_{i}.jpg')
        except Exception as e:
            print(f"Error creating dummy image {i}: {e}")
    print(f"Created dummy image files in: {dummy_image_dir}")

    # 3. Load the benchmark data
    df_benchmark = load_benchmark_data(benchmark_name, benchmark_path)

    if df_benchmark is not None:
        print(f"\nLoaded benchmark data:\n{df_benchmark.head()}")

        # 4. Define class names for this benchmark
        class_names = ["class_a", "class_b", "class_c"] # Must match the labels in the data

        # 5. Create Zero-Shot DataLoader
        zs_dl = create_zeroshot_dataloader(
            df=df_benchmark,
            benchmark_base_path=(benchmark_path / benchmark_name), # Path where images are relative to
            batch_size=4,
            num_workers=0 # Use 0 for debugging ease
        )

        if zs_dl:
            print(f"\nSuccessfully created Zero-Shot DataLoader.")
            # Inspect a batch
            print("Inspecting one batch from Zero-Shot DataLoader:")
            batch = zs_dl.one_batch()
            if isinstance(batch, tuple) and len(batch) == 2:
                img_batch, lbl_batch = batch
                print(f"  Image Batch Shape: {img_batch.shape}")
                print(f"  Label Batch Shape: {lbl_batch.shape}")
                print(f"  Label Batch Values: {lbl_batch}")
                assert img_batch.ndim == 4
                assert lbl_batch.ndim == 1
            else:
                print(f"Unexpected batch format: {type(batch)}")
        else:
            print("\nFailed to create Zero-Shot DataLoader.")

    else:
        print(f"\nFailed to load benchmark data for '{benchmark_name}'.")

    # Clean up dummy files (optional)
    # import shutil
    # if (benchmark_path / benchmark_name).exists():
    #     shutil.rmtree(benchmark_path / benchmark_name)
    #     print(f"\nCleaned up dummy benchmark directory: {benchmark_path / benchmark_name}")

    print("\n--- Example Finished ---")

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export() # Run this in terminal to export